In [2]:
import networkx as nx
from networkx.algorithms import isomorphism
import numpy as np
from google.colab import drive
import community
import matplotlib.pyplot as plt
import os
import sys

drive_folder_path = r'/content/drive/MyDrive/UNI/magistrale/LearningFromNetworks/datasets'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# tool-graph installation
# python-louvain installation

!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

# Colab uses a Python install that deviates from the system's! Bad collab! We need some workarounds.
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

# Install necessary libraries
!pip install pynauty
!pip install python-louvain

Executing: /tmp/apt-key-gpghome.NuZLTQwVLD/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
gpg: key 612DEFB798507F25: "Tiago de Paula Peixoto <tiago@skewed.de>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://downloads.skewed.de/apt jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https:

In [5]:
import graph_tool.all as gt

#1.c - Compute the clustering density and the modularity of the network (point 1.c)


Verify and quantify the tendency of each group to communicate inside itself (evaluating also
the presence of an internal hierarchy) or with the other groups; we will cover this point by
Computing the clustering coefficient, the modularity of the network and/or clustering density.


In [16]:
def print_nodes_and_communities(G, communities):
    for node, community in zip(communities, G.nodes()):
        print(f"Community: {community}, Node: {node}")


def read_network_from_file(file_path):
    G = nx.Graph()
    with open(file_path, "r") as file:
      for line in file:
          parts = line.split()
          node1 = int(parts[0])
          node2 = int(parts[1])
          weight = float(parts[2])
          G.add_edge(node1, node2, weight=weight)
    return G

def loadNetwork(file_name):
      file_path = os.path.join(drive_folder_path, file_name)
      graph = read_network_from_file(file_path)
      return graph

def calculate_modularity(G, communities):
    modularity = nx.community.modularity(G, communities)
    return modularity

def cluster_density_communities(G, communities):
  for i, community in enumerate(communities):
    subgraph = G.subgraph(community)
    clustering_density = nx.algorithms.cluster.average_clustering(subgraph)
    print(f"Community {i+1} Clustering Density: {clustering_density}")

In [18]:
# Get a list of all dataset names in the drive folder folder

file_list = os.listdir(drive_folder_path)

file_list.sort()

print(file_list)

#visualize_networks(all_graph, file_list)



['Colony1-day01.txt', 'Colony1-day05.txt', 'Colony1-day10.txt', 'Colony2-day01.txt', 'Colony2-day05.txt', 'Colony2-day10.txt', 'Colony3-day01.txt', 'Colony3-day05.txt', 'Colony3-day09.txt']


In [19]:
for fine_name in file_list:
  print(fine_name)
  # Step 1: Read the network from the file
  G = loadNetwork(fine_name)

  # Step 2: Detect communities
  communities = nx.community.louvain_communities(G)

  # Step 3: Calculate modularity
  modularity = calculate_modularity(G, communities)

  # Compute clustering density
  clustering_density = nx.density(G)

  #print
  print(f"Colony in file: {fine_name}")

  print(f"Modularity: {modularity}")

  # can be uncommented to visualize association community-nodes
  #print_nodes_and_communities(G, communities)

  print(f"Clustering Density: {clustering_density}")

  print(f"Clustering Density Community:")
  clustering_density_community = cluster_density_communities(G, communities)

  print(f"communities: {communities}")

  print(f"-------------------------------------------------------------------")

Colony1-day01.txt
Colony in file: Colony1-day01.txt
Modularity: 0.3098593085947753
Clustering Density: 0.7190265486725663
Clustering Density Community:
Community 1 Clustering Density: 0.924689193492161
Community 2 Clustering Density: 0.9493589743589744
Community 3 Clustering Density: 0.9766534658288794
communities: [{3, 4, 12, 13, 14, 15, 20, 21, 23, 25, 27, 30, 31, 33, 35, 37, 41, 43, 48, 51, 55, 62, 66, 69, 72, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110}, {64, 67, 6, 8, 10, 24, 29, 94, 32, 40, 111, 112, 113, 50, 60, 61}, {1, 2, 5, 7, 9, 11, 16, 17, 18, 19, 22, 26, 28, 34, 36, 38, 39, 42, 44, 45, 46, 47, 49, 52, 53, 54, 56, 57, 58, 59, 63, 65, 68, 70, 71, 73, 74, 75, 76, 77, 79}]
-------------------------------------------------------------------
Colony1-day05.txt
Colony in file: Colony1-day05.txt
Modularity: 0.3240769348980034
Clustering Density: 0.6556573957016435
Clustering Density Community:

# 1.c custom functions
(not used)

In [59]:
# Clustering density

# AKA Global clustering-coefficient

# Algorithm calculate_clustering_density(dictionary):
# Input: Dictionary where keys are nodes, the values are the corresponding clustering coefficients
# Output: value representing the clustering density / global clustering coefficient

def my_calculate_clustering_density(clustering_coefficients):

    num_nodes = len(clustering_coefficients)

    if num_nodes == 0:
        return 0.0

    clustering_density = sum(clustering_coefficients.values()) / num_nodes
    return clustering_density


In [60]:
# Adjacency matrix

# Algorithm create_adjacency_matrix(num_nodes, edges):
# Input: number of nodes in the network and a list of edges represented as pairs of nodes
# Output: square matrix representing a graph

def my_create_adjacency_matrix(num_nodes, edges):
    # Initialize an empty adjacency matrix with zeros
    adjacency_matrix = [[0] * num_nodes for _ in range(num_nodes)]

    # Populate the adjacency matrix based on the edges
    for edge in edges:
        node1, node2 = edge
        # Assuming an undirected graph, so we set both entries to 1
        adjacency_matrix[node1][node2] = 1
        adjacency_matrix[node2][node1] = 1

    return adjacency_matrix

In [63]:
# Modularity of the network


# Algorithm calculate_modularity(community_assignments , edges):
# Input: "community_assignments" is a list containing the community assignment for each node. "edges" is a list of edges represented as pairs of nodes
# Output: modularity of the network: -1 <= modularity <= 1

def my_calculate_modularity(community_assignments , edges):

    adjacency_matrix = my_create_adjacency_matrix(len(community_assignments), edges)

    num_nodes = len(adjacency_matrix)
    num_edges = sum(sum(row) for row in adjacency_matrix) // 2

    modularity = 0.0

    for i in range(num_nodes):
        for j in range(num_nodes):
            A_ij = adjacency_matrix[i][j]
            k_i = sum(adjacency_matrix[i])
            k_j = sum(adjacency_matrix[j])
            delta_c = 1 if community_assignments[i] == community_assignments[j] else 0

            modularity += (A_ij - (k_i * k_j) / (2 * num_edges)) * delta_c

    modularity /= 2 * num_edges  # Normalize by twice the number of edges

    return modularity

#3.a - Discover frequent subgraphs using Frequent Subgraph Growth algorithm (point 3.a)


Pattern recognition: see if there are any particular configurations repeating in the various networks;

------------------------------------------------------------

*In this section i tried to implement a basic FSA (Frequent Subgraph discovery Algorithm) algorithm, without success as no common subgraph was found*

Tried to implement a simple FSA (Frequent subgraph discovery algorithm). Wihout success

In [6]:
def initialize_candidates(graph):
    # Generate initial single nodes (candidates)
    candidates = []
    for node in graph:
        candidates.append(([node], []))
    return candidates

def generate_candidates(current_candidates, graph):
    new_candidates = []
    for candidate, edges in current_candidates:
        # Expand the candidate to create new candidates
        for neighbor in graph[candidate[-1]]:
            if neighbor not in candidate:
                new_candidates.append((candidate + [neighbor], edges + [(candidate[-1], neighbor)]))
    return new_candidates

def count_support(candidates, dataset):
    support_count = {}

    # If dataset is a single graph, wrap it in a list
    if not isinstance(dataset, list):
        dataset = [dataset]

    for candidate, edges in candidates:
        for graph_edges in dataset:
            # Convert edges to a set to handle both integer and tuple cases
            graph_edges_set = set(graph_edges) if isinstance(graph_edges, (list, tuple)) else set([graph_edges])
            if all(edge in graph_edges_set for edge in edges):
                key = tuple(candidate)
                if key in support_count:
                    support_count[key] += 1
                else:
                    support_count[key] = 1

    # Include all possible candidates in the support_count dictionary
    for candidate, _ in candidates:
        key = tuple(candidate)
        if key not in support_count:
            support_count[key] = 0

    return support_count


def prune_candidates(candidates, support_count, min_support):
    pruned_candidates = [(candidate, edges) for candidate, edges in candidates if support_count[tuple(candidate)] >= min_support]
    return pruned_candidates

def visualize_graph(graph_edges):
    # Create a NetworkX graph and add edges
    G = nx.Graph()
    G.add_edges_from(graph_edges)

    # Visualize the graph
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='skyblue', node_size=100, font_size=8)
    plt.show()

def read_network_from_file_ex3(file_path):
    G = nx.Graph()
    with open(file_path, "r") as file:
      for line in file:
          parts = line.split()
          node1 = int(parts[0])
          node2 = int(parts[1])
          weight = float(parts[2])
          G.add_edge(node1, node2, weight=weight)
    return G


def mine_frequent_subgraphs(all_datasets, min_support):
    for dataset in all_datasets:
        candidates = initialize_candidates(dataset)
        while candidates:
            # Generate and expand candidates
            candidates = generate_candidates(candidates, dataset)

            # Count support
            support_count = count_support(candidates, dataset)

            # Prune candidates based on minimum support
            candidates = prune_candidates(candidates, support_count, min_support)

            # Output frequent subgraphs or store them as needed
            print("Frequent Subgraphs:", [tuple(candidate) for candidate, _ in candidates])

            for candidate, edges in candidates:
                print("Frequent Subgraph:", tuple(candidate))
                visualize_graph(edges)

def mine_frequent_subgraphs_debug(all_datasets, min_support):
    iteration = 1
    for dataset in all_datasets:
        candidates = initialize_candidates(dataset)
        while candidates:
            print(f"Iteration {iteration}")

            # Generate and expand candidates
            candidates = generate_candidates(candidates, dataset)
            print("Generated Candidates:", candidates)

            # Count support
            support_count = count_support(candidates, dataset)
            print("Support Count:", support_count)

            # Prune candidates based on minimum support
            candidates = prune_candidates(candidates, support_count, min_support)
            print("Pruned Candidates:", candidates)

            # Output frequent subgraphs or store them as needed
            print("Frequent Subgraphs:", [tuple(candidate) for candidate, _ in candidates])

            for candidate, edges in candidates:
                print("Frequent Subgraph:", tuple(candidate))
                visualize_graph(edges)

            iteration += 1

In [7]:
# List all files in the folder
file_list = os.listdir(drive_folder_path)

# Create an empty list to store datasets
all_datasets = []

# Iterate through each file
for file_name in file_list:
    # Construct the full path to the file
    file_path = os.path.join(drive_folder_path, file_name)

    # Read the graph from the file and append to the list
    dataset = read_network_from_file_ex3(file_path)

    all_datasets.append(dataset)

In [8]:
# Execute frequent subgraph mining for all datasets
min_support = 2
mine_frequent_subgraphs_debug(all_datasets, min_support)


Iteration 1
Generated Candidates: [([1, 81], [(1, 81)]), ([1, 52], [(1, 52)]), ([1, 66], [(1, 66)]), ([1, 75], [(1, 75)]), ([1, 67], [(1, 67)]), ([1, 6], [(1, 6)]), ([1, 7], [(1, 7)]), ([1, 54], [(1, 54)]), ([1, 10], [(1, 10)]), ([1, 11], [(1, 11)]), ([1, 14], [(1, 14)]), ([1, 15], [(1, 15)]), ([1, 8], [(1, 8)]), ([1, 18], [(1, 18)]), ([1, 19], [(1, 19)]), ([1, 89], [(1, 89)]), ([1, 20], [(1, 20)]), ([1, 22], [(1, 22)]), ([1, 92], [(1, 92)]), ([1, 24], [(1, 24)]), ([1, 21], [(1, 21)]), ([1, 28], [(1, 28)]), ([1, 29], [(1, 29)]), ([1, 5], [(1, 5)]), ([1, 31], [(1, 31)]), ([1, 76], [(1, 76)]), ([1, 33], [(1, 33)]), ([1, 34], [(1, 34)]), ([1, 35], [(1, 35)]), ([1, 95], [(1, 95)]), ([1, 36], [(1, 36)]), ([1, 53], [(1, 53)]), ([1, 38], [(1, 38)]), ([1, 98], [(1, 98)]), ([1, 99], [(1, 99)]), ([1, 39], [(1, 39)]), ([1, 40], [(1, 40)]), ([1, 41], [(1, 41)]), ([1, 42], [(1, 42)]), ([1, 43], [(1, 43)]), ([1, 44], [(1, 44)]), ([1, 45], [(1, 45)]), ([1, 46], [(1, 46)]), ([1, 47], [(1, 47)]), ([1, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# 3.a.1 - Further test for 3.a point

In this section main challenge i faceed was the computational and space limitations of the free version of Colab.

In this section i started experimenting some pre-build function that allow to find if a graph is subgraph of another (The main problem is that in this way it cannot be found any frequent subgraph: we could know only if graphs G1 and G2 are not isomorphic)

In [5]:
# read files in drive
def read_network_from_file(file_path):
    G = nx.Graph()
    with open(file_path, "r") as file:
      for line in file:
          parts = line.split()
          node1 = int(parts[0])
          node2 = int(parts[1])
          weight = float(parts[2])
          G.add_edge(node1, node2, weight=weight)
    return G

# print networks to be visualized
def visualize_networks(networks, file_list):
  for network, file_name in zip(networks, file_list):
      pos = nx.spring_layout(network)
      nx.draw(network, pos, with_labels=True, font_weight='bold', node_size=500, node_color='skyblue')
      plt.title(f'Network Visualization - {file_name}')
      plt.show()

# load networks in array
def loadNetworks(file_list):
  all_graph = []
  # Iterate through each file
  for file_name in file_list:
      # Construct the full path to the file
      file_path = os.path.join(drive_folder_path, file_name)

      # Read the graph from the file and append to the list
      dataset = read_network_from_file(file_path)

      all_graph.append(dataset)
  return all_graph


In [43]:
# List all files in the folder
file_list = os.listdir(drive_folder_path)

file_list.sort()

print(file_list)

all_graph = loadNetworks(file_list)

#visualize_networks(all_graph, file_list)

['Colony1-day01.txt', 'Colony1-day05.txt', 'Colony1-day10.txt', 'Colony2-day01.txt', 'Colony2-day05.txt', 'Colony2-day10.txt', 'Colony3-day01.txt', 'Colony3-day05.txt', 'Colony3-day09.txt']


In [ ]:
# Returns False if graphs are not isomorphic.
# Requires ~2hrs
print("could_be_isomorphic(G1, G2) ?")
for i in range(0, len(all_graph)//3 -1, 1):
    temp = nx.could_be_isomorphic(all_graph[i], all_graph[i+1])
    print(f"Iteration number {i//2 + 1}: Nodes from graphs {i+1} and {i+2} controlled: Result is {temp}")

In [ ]:
# Requires ~2hrs
for i in range(3, 3+ len(all_graph)//3 -1, 1):
    temp = nx.could_be_isomorphic(all_graph[i], all_graph[i+1])
    print(f"Iteration number {i//2 + 1}: Nodes from graphs {i+1} and {i+2} controlled: Result is {temp}")

Iteration number 2: Nodes from graphs 4 and 5 controlled: Result is False
Iteration number 3: Nodes from graphs 5 and 6 controlled: Result is False


In [ ]:
# Colab ram constraints limit the computation of this loop
for i in range(6, 6 + len(all_graph)//3 -1, 1):
    temp = nx.could_be_isomorphic(all_graph[i], all_graph[i+1])
    print(f"Iteration number {i//2 + 1}: Nodes from graphs {i+1} and {i+2} controlled: Result is {temp}")

In [ ]:
print("fast_could_be_isomorphic(G1, G2) ?")
for i in range(len(all_graph)-1):
  temp = nx.fast_could_be_isomorphic(all_graph[i], all_graph[i+1])
  print(f"itaration number {i}: nodes from graphs {i+1} and {i+2} controlled: result is {temp}")

fast_could_be_isomorphic(G1, G2) ?
itaration number 0: nodes from graphs 1 and 2 controlled: result is False
itaration number 1: nodes from graphs 2 and 3 controlled: result is False
itaration number 2: nodes from graphs 3 and 4 controlled: result is False
itaration number 3: nodes from graphs 4 and 5 controlled: result is False
itaration number 4: nodes from graphs 5 and 6 controlled: result is False
itaration number 5: nodes from graphs 6 and 7 controlled: result is False
itaration number 6: nodes from graphs 7 and 8 controlled: result is False
itaration number 7: nodes from graphs 8 and 9 controlled: result is False


In [ ]:
print("faster_could_be_isomorphic(G1, G2) ?")
for i in range(len(all_graph)-1):
  temp = nx.faster_could_be_isomorphic(all_graph[i], all_graph[i+1])
  print(f"itaration number {i}: nodes from graphs {i+1} and {i+2} controlled: result is {temp}")

faster_could_be_isomorphic(G1, G2) ?
itaration number 0: nodes from graphs 1 and 2 controlled: result is False
itaration number 1: nodes from graphs 2 and 3 controlled: result is False
itaration number 2: nodes from graphs 3 and 4 controlled: result is False
itaration number 3: nodes from graphs 4 and 5 controlled: result is False
itaration number 4: nodes from graphs 5 and 6 controlled: result is False
itaration number 5: nodes from graphs 6 and 7 controlled: result is False
itaration number 6: nodes from graphs 7 and 8 controlled: result is False
itaration number 7: nodes from graphs 8 and 9 controlled: result is False


In [7]:
all_tree = []

for graph in all_graph:
  tree = nx.minimum_spanning_tree(graph)
  all_tree.append(tree)
  #plt.subplot(122)
  #nx.draw(tree, with_labels=True, font_weight='bold', node_color='lightcoral', font_color='black')
  #plt.title("Minimum Spanning Tree")

In [33]:
for i in range(len(all_tree)-2):
  temp = isomorphism.tree_isomorphism(all_tree[i], all_tree[i+1])
  print(f"{temp}")

# no matches found

[]
[]
[]
[]
[]
[]
[]


In [9]:
result = []
result_iter = []
for i in range(len(all_graph)-1):
  matcher = isomorphism.GraphMatcher(all_graph[i], all_graph[i+1])
  #result.append(matcher.subgraph_is_isomorphic())
  #print(result[i])
  result_iter.append(matcher.subgraph_isomorphisms_iter())

# This possible solution cannot be used because of computational issue (Colab stop computation for high ram usage)

The following solution take too long to execute (colab has ended the session)

In [10]:
def find_subgraph_matches(graph1, graph2):
    matches = isomorphism.GraphMatcher(graph1, graph2)
    matches = matches.subgraph_is_isomorphic()
    return matches


#3.a.2 - In this last subsection i tried to optimize the process by transitioning from searching for frequent subgraphs and subgraphs directly from the entire graphs to seeking common subgraphs by dividing the colony into smaller graphs

This approach capitalizes on the knowledge that these subgraphs contain clusters of ants, with the goal of identifying recurrent patterns within these more manageable subgraphs.

In [9]:
def read_network_from_file(file_path):
    G = nx.Graph()
    with open(file_path, "r") as file:
      for line in file:
          parts = line.split()
          node1 = int(parts[0])
          node2 = int(parts[1])
          weight = float(parts[2])
          G.add_edge(node1, node2, weight=weight)
    return G

def loadNetworks(file_list):
    all_graph = []
    # Iterate through each file
    for file_name in file_list:
      # Construct the full path to the file
      file_path = os.path.join(drive_folder_path, file_name)
      # Read the graph from the file and append to the list
      dataset = read_network_from_file(file_path)
      all_graph.append(dataset)
    return all_graph


def convert_nx_to_gt(nx_graph):
    gt_graph = gt.Graph(directed=nx_graph.is_directed())

    # Map from NetworkX nodes to graph-tool vertices
    nx_to_gt_vertex = {}

    # Add vertices to the graph
    for node in nx_graph.nodes():
        v = gt_graph.add_vertex()
        nx_to_gt_vertex[node] = v

    # Add edges to the graph
    for edge in nx_graph.edges():
        source = nx_to_gt_vertex[edge[0]]
        target = nx_to_gt_vertex[edge[1]]
        gt_graph.add_edge(source, target)

    return gt_graph

def separate_subsets(result):
    # Initialize three lists to store subsets
    if(len(result) == 2):
      return result[0], result[1], []
    return result[0], result[1], result[2]

In [10]:
# List all files in the folder
file_names = os.listdir(drive_folder_path)

file_names.sort()

all_graph = loadNetworks(file_names)
print(file_names)
# divided community
communities1 = []
communities2 = []
communities3 = []

for graph in all_graph:
  # Step 2: Detect communities
  lv_res = nx.community.louvain_communities(graph)
  temp1, temp2, temp3 = separate_subsets(lv_res)
  communities1.append(temp1)
  communities2.append(temp2)
  communities3.append(temp3)

['Colony1-day01.txt', 'Colony1-day05.txt', 'Colony1-day10.txt', 'Colony2-day01.txt', 'Colony2-day05.txt', 'Colony2-day10.txt', 'Colony3-day01.txt', 'Colony3-day05.txt', 'Colony3-day09.txt']


In [11]:

for i in range(0,9):
  print(f"ID: {0 + 3*i}, Community: {communities1[i]}")
  print(f"ID: {1 + 3*i}, Community: {communities2[i]}")
  print(f"ID: {2 + 3*i}, Community: {communities3[i]}")


ID: 0, Community: {3, 4, 12, 13, 14, 15, 20, 21, 23, 25, 27, 30, 31, 33, 35, 37, 41, 43, 48, 51, 55, 62, 66, 69, 72, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110}
ID: 1, Community: {1, 2, 5, 7, 9, 11, 16, 17, 18, 19, 22, 26, 28, 34, 36, 38, 39, 42, 44, 45, 46, 47, 49, 52, 53, 54, 56, 57, 58, 63, 65, 68, 70, 71, 73, 74, 75, 76, 77, 79}
ID: 2, Community: {64, 67, 6, 8, 10, 24, 29, 94, 32, 40, 111, 112, 113, 50, 59, 60, 61}
ID: 3, Community: {1, 2, 7, 9, 76, 77, 16, 22, 28, 32, 34, 47, 52, 53, 56, 57, 63}
ID: 4, Community: {5, 6, 8, 10, 11, 17, 18, 19, 29, 36, 38, 39, 42, 44, 45, 46, 49, 50, 54, 58, 59, 60, 65, 68, 70, 71, 73, 74, 75, 79, 89, 98, 111, 113}
ID: 5, Community: {3, 4, 12, 13, 14, 15, 20, 21, 23, 24, 25, 26, 27, 30, 31, 33, 35, 37, 40, 41, 43, 48, 51, 55, 61, 62, 64, 66, 67, 69, 72, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105, 10

In [12]:
# create graph of communities
all_communities_graph = []

for i in range(len(all_graph)-1):
  community_graph1 = all_graph[i].subgraph(communities1[i])
  community_graph2 = all_graph[i].subgraph(communities2[i])
  community_graph3 = all_graph[i].subgraph(communities3[i])

  all_communities_graph.append(community_graph1)
  all_communities_graph.append(community_graph2)
  all_communities_graph.append(community_graph3)

# saved format:
# communitiyx = {...} {...}

In [70]:
# create graph from communities
# in all_communities_graph the graphs are stored as: community1 graph 1, community2 graph 1, community3 graph 1, community1 graph 2....

# ID - Coomunity # Membership # - Colony # Day #
# 0   c1g1 ┐
# 1   c2g1 | c1d1
# 2   c3g1 ┘
# 3   c1g2 ┐
# 4   c2g2 | c1d5
# 5   c3g2 ┘
# 6   c1g3 ┐
# 7   c2g3 | c1d10
# 8   c3g3 ┘
# 9   c1g4 ┐
# 10  c2g4 | c2d1
# 11  c3g4 ┘
# 12  c1g5 ┐
# 13  c2g5 | c2d5
# 14  c3g5 ┘
# 15  c1g6 ┐
# 16  c2g6 | c2d10
# 17  c3g6 ┘
# 18  c1g7 ┐
# 19  c2g7 | c3d1
# 20  c3g7 ┘
# 21  c1g8 ┐
# 22  c2g8 | c3d5
# 23  c3g8 ┘
# 24  c1g9 ┐
# 25  c2g9 | c3d9
# 26  c3g9 ┘


## Colony 1: checked community 1 in days 1 and 5

In [78]:
# Graph in which the search is performed
# Community0 -> Community1 day1
graph = convert_nx_to_gt(all_communities_graph[0])

# Subset of elements of communities - this is done due to colab computational
# Resources restrictions in free plan

elements_subset = communities1[1] # in communities list 1 take the second list available -> community 1 day 5

# Changed values: [:10], [10:30]
sub = list(elements_subset)[:10]

temp_graph= all_graph[1].subgraph(sub)

subgraph = convert_nx_to_gt(temp_graph)


# 1 param: graph to be searched, 2 param graph in which the search is performed
returnList = gt.subgraph_isomorphism(subgraph,graph,max_n=1)

print(returnList)

# Create a graph drawing object
pos = gt.sfdp_layout(graph)

# Plot the original graph
gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output="original_graph.png")

counter = 0

# Plot the subgraph matches
for match in returnList:
    counter = counter + 1
    subgraph_copy = gt.Graph(subgraph)

    # Create a new vertex property map for the subgraph_copy
    subgraph_copy.vp["vertex_map"] = subgraph_copy.new_vertex_property("int")

    for v in graph.vertices():
        if v in match:
            subgraph_copy.vp["vertex_map"][v] = match[v]

    # Highlight the matched subgraph in the original graph
    gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output=f"matched_subgraph_.png", vprops={"vertex_map": subgraph_copy.vp["vertex_map"]})


[<VertexPropertyMap object with value type 'int64_t', for Graph 0x7c8314fa1990, at 0x7c8314fa1720>]


In [25]:
# Graph in which the search is performed
# community0 -> community1 day5
graph = convert_nx_to_gt(all_communities_graph[3])

# subset of elements of communities - this is done due to colab computational
# resources restrictions in free plan

elements_subset = communities1[2] # in communities list 1 take the second list available -> community 1 day 10

# changed values: [:10], [10:30]
sub = list(elements_subset)[:10]

temp_graph= all_graph[1].subgraph(sub)

subgraph = convert_nx_to_gt(temp_graph)


# 1 param: graph to be searched, 2 param graph in which the search is performed
returnList = gt.subgraph_isomorphism(subgraph,graph,max_n=1)

print(returnList)

# Create a graph drawing object
pos = gt.sfdp_layout(graph)

# Plot the original graph
gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output="original_graph.png")

counter = 0

# Plot the subgraph matches
for match in returnList:
    counter = counter + 1
    subgraph_copy = gt.Graph(subgraph)

    # Create a new vertex property map for the subgraph_copy
    subgraph_copy.vp["vertex_map"] = subgraph_copy.new_vertex_property("int")

    for v in graph.vertices():
        if v in match:
            subgraph_copy.vp["vertex_map"][v] = match[v]

    # Highlight the matched subgraph in the original graph
    gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output=f"matched_subgraph_.png", vprops={"vertex_map": subgraph_copy.vp["vertex_map"]})


[<VertexPropertyMap object with value type 'int64_t', for Graph 0x7f20a356d5d0, at 0x7f20599d1390>]


##Colony 1: checked community 2 in days 5 and 10

In [88]:
# Graph in which the search is performed
# community4 -> community2 day5
graph = convert_nx_to_gt(all_communities_graph[4])

# subset of elements of communities - this is done due to colab computational
# resources restrictions in free plan

elements_subset = communities2[2] # in communities list 2 take the third list available -> community 1 day 5

# changed values: [:10], [10:30],
sub = list(elements_subset)[:10]

temp_graph= all_graph[1].subgraph(sub)

subgraph = convert_nx_to_gt(temp_graph)


# 1 param: graph to be searched, 2 param graph in which the search is performed
returnList = gt.subgraph_isomorphism(subgraph,graph,max_n=1)

print(returnList)

# Create a graph drawing object
pos = gt.sfdp_layout(graph)

# Plot the original graph
gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output="original_graph.png")

counter = 0

# Plot the subgraph matches
for match in returnList:
    counter = counter + 1
    subgraph_copy = gt.Graph(subgraph)

    # Create a new vertex property map for the subgraph_copy
    subgraph_copy.vp["vertex_map"] = subgraph_copy.new_vertex_property("int")

    for v in graph.vertices():
        if v in match:
            subgraph_copy.vp["vertex_map"][v] = match[v]

    # Highlight the matched subgraph in the original graph
    gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output=f"matched_subgraph_.png", vprops={"vertex_map": subgraph_copy.vp["vertex_map"]})


[<VertexPropertyMap object with value type 'int64_t', for Graph 0x7c8314115f60, at 0x7c8314117ee0>]


##Colony 2: checked community 3 in days 1 and 5

In [89]:
# Graph in which the search is performed
# community4 -> community2 day5
graph = convert_nx_to_gt(all_communities_graph[11])

# subset of elements of communities - this is done due to colab computational
# resources restrictions in free plan

elements_subset = communities3[4] # in communities list 3 take the fourth el. in list -> community 3 day 5

# changed values: [:10], [10:30],
sub = list(elements_subset)[10:30]

temp_graph= all_graph[1].subgraph(sub)

subgraph = convert_nx_to_gt(temp_graph)


# 1 param: graph to be searched, 2 param graph in which the search is performed
returnList = gt.subgraph_isomorphism(subgraph,graph,max_n=1)

print(returnList)

# Create a graph drawing object
pos = gt.sfdp_layout(graph)

# Plot the original graph
gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output="original_graph.png")

counter = 0

# Plot the subgraph matches
for match in returnList:
    counter = counter + 1
    subgraph_copy = gt.Graph(subgraph)

    # Create a new vertex property map for the subgraph_copy
    subgraph_copy.vp["vertex_map"] = subgraph_copy.new_vertex_property("int")

    for v in graph.vertices():
        if v in match:
            subgraph_copy.vp["vertex_map"][v] = match[v]

    # Highlight the matched subgraph in the original graph
    gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output=f"matched_subgraph_.png", vprops={"vertex_map": subgraph_copy.vp["vertex_map"]})


[<VertexPropertyMap object with value type 'int64_t', for Graph 0x7c836abfe020, at 0x7c8314114dc0>]


##Colony 1: checked community 2 in days 1 - 5 and 5 - 10





In [24]:
# Graph in which the search is performed
# community1 -> community2 day1
graph = convert_nx_to_gt(all_communities_graph[1])

# subset of elements of communities - this is done due to colab computational
# resources restrictions in free plan

elements_subset = communities2[1] # in communities list 2 take the first el. in list -> community 2 day 5


sub = list(elements_subset)[10:15]

temp_graph= all_graph[1].subgraph(sub)

subgraph = convert_nx_to_gt(temp_graph)


# 1 param: graph to be searched, 2 param graph in which the search is performed
returnList = gt.subgraph_isomorphism(subgraph,graph,max_n=1)

print(returnList)

# Create a graph drawing object
pos = gt.sfdp_layout(graph)

# Plot the original graph
gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output="original_graph.png")

counter = 0

# Plot the subgraph matches
for match in returnList:
    counter = counter + 1
    subgraph_copy = gt.Graph(subgraph)

    # Create a new vertex property map for the subgraph_copy
    subgraph_copy.vp["vertex_map"] = subgraph_copy.new_vertex_property("int")

    for v in graph.vertices():
        if v in match:
            subgraph_copy.vp["vertex_map"][v] = match[v]

    # Highlight the matched subgraph in the original graph
    gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output=f"matched_subgraph_.png", vprops={"vertex_map": subgraph_copy.vp["vertex_map"]})

[<VertexPropertyMap object with value type 'int64_t', for Graph 0x7f20599d2b90, at 0x7f20599d3e80>]


In [20]:
# Graph in which the search is performed
# community4 -> community2 day10
graph = convert_nx_to_gt(all_communities_graph[4])

# subset of elements of communities - this is done due to colab computational
# resources restrictions in free plan

elements_subset = communities2[2] # in communities list 2 take the third el. in list -> community 2 day 10


sub = list(elements_subset)[10:25]

temp_graph= all_graph[1].subgraph(sub)

subgraph = convert_nx_to_gt(temp_graph)


# 1 param: graph to be searched, 2 param graph in which the search is performed
returnList = gt.subgraph_isomorphism(subgraph,graph,max_n=1)

print(returnList)

# Create a graph drawing object
pos = gt.sfdp_layout(graph)

# Plot the original graph
gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output="original_graph.png")

counter = 0

# Plot the subgraph matches
for match in returnList:
    counter = counter + 1
    subgraph_copy = gt.Graph(subgraph)

    # Create a new vertex property map for the subgraph_copy
    subgraph_copy.vp["vertex_map"] = subgraph_copy.new_vertex_property("int")

    for v in graph.vertices():
        if v in match:
            subgraph_copy.vp["vertex_map"][v] = match[v]

    # Highlight the matched subgraph in the original graph
    gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output=f"matched_subgraph_.png", vprops={"vertex_map": subgraph_copy.vp["vertex_map"]})

[<VertexPropertyMap object with value type 'int64_t', for Graph 0x7f20599d1ed0, at 0x7f20599d35b0>]


##Colony 1: checked community 3 queen search

Assuming queen is idenified by node with ID:66



In [13]:
# Graph in which the search is performed
# community4 -> community2 day5
graph = convert_nx_to_gt(all_communities_graph[4])

# subset of elements of communities - this is done due to colab computational
# resources restrictions in free plan

# communities2 is kept because louvian algorithm sometimes change the clusters
elements_subset = communities3[0] # in communities list 3 take the second el. in list -> community 3 day 5


sub = list(elements_subset)[10:25]
print(sub)
temp_graph= all_graph[1].subgraph(sub)

subgraph = convert_nx_to_gt(temp_graph)


# 1 param: graph to be searched, 2 param graph in which the search is performed
returnList = gt.subgraph_isomorphism(subgraph,graph,max_n=1)

print(returnList)

# Create a graph drawing object
pos = gt.sfdp_layout(graph)

# Plot the original graph
gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output="original_graph.png")

counter = 0

# Plot the subgraph matches
for match in returnList:
    counter = counter + 1
    subgraph_copy = gt.Graph(subgraph)

    # Create a new vertex property map for the subgraph_copy
    subgraph_copy.vp["vertex_map"] = subgraph_copy.new_vertex_property("int")

    for v in graph.vertices():
        if v in match:
            subgraph_copy.vp["vertex_map"][v] = match[v]

    # Highlight the matched subgraph in the original graph
    gt.graph_draw(graph, pos, output_size=(800, 600), vertex_text=graph.vertex_index, vertex_font_size=12, edge_font_size=10, output=f"matched_subgraph_only_queen{counter}.png", vprops={"vertex_map": subgraph_copy.vp["vertex_map"]})

[111, 112, 113, 50, 59, 60, 61]
[<VertexPropertyMap object with value type 'int64_t', for Graph 0x7aeeb042dc00, at 0x7aeeae72a4a0>]


/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:519: UserWarning: Unknown attribute: vertex_map
  warnings.warn("Unknown attribute: " + str(k), UserWarning)
